## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 2nd Feb. 9th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,February-3-2025,Condo/Co-op,1600 SW 1st Ave #814,Miami,FL,33129.0,635000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1600-SW-1st-Av...,MARMLS,A11587630,N,Y,25.759352,-80.197611


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [9]:
df2 = df

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-07    24
2025-02-03    20
2025-02-05    18
2025-02-06    11
2025-02-04     8
2025-02-10     3
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
79,PAST SALE,2025-02-05,Condo/Co-op,9111 Collins Ave Unit N-611,Surfside,FL,33154.0,7800000.0,1.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9111-Collin...,MARMLS,A11580902,N,Y,25.878108,-80.121454
80,PAST SALE,2025-02-06,Condo/Co-op,5959 Collins Ave #1803,Miami Beach,FL,33140.0,6190000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11513873,N,Y,25.841880,-80.120969
8,PAST SALE,2025-02-04,Condo/Co-op,788 NE 23rd St #5602,Miami,FL,33137.0,4500000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,MARMLS,A11738937,N,Y,25.799065,-80.185542
21,PAST SALE,2025-02-07,Condo/Co-op,1435 Brickell Ave #3501,Miami,FL,33131.0,2895000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1435-Brickell-...,MARMLS,A11715454,N,Y,25.759193,-80.191943
43,PAST SALE,2025-02-04,Condo/Co-op,15901 Collins Ave #2506,Sunny Isles Beach,FL,33160.0,2090000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11694322,N,Y,25.923693,-80.121772
28,PAST SALE,2025-02-03,Condo/Co-op,17001 Collins Ave #4107,Sunny Isles Beach,FL,33160.0,2060000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11695000,N,Y,25.933531,-80.120941
49,PAST SALE,2025-02-07,Condo/Co-op,1331 Brickell Bay Dr #4303,Miami,FL,33131.0,1900000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11666572,N,Y,25.760000,-80.189163
83,PAST SALE,2025-02-03,Condo/Co-op,700 Coral Way #3,Coral Gables,FL,33134.0,1800000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/700-Cor...,MARMLS,A11526899,N,Y,25.748811,-80.269144
56,PAST SALE,2025-02-06,Condo/Co-op,495 Brickell Ave #3101,Miami,FL,33131.0,1600000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/495-Brickell-A...,MARMLS,A11668802,N,Y,25.768352,-80.188672
62,PAST SALE,2025-02-03,Condo/Co-op,650 NE 32nd St #4001,Miami,FL,33137.0,1575000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/650-NE-32nd-St...,MARMLS,A11664569,N,Y,25.806902,-80.186249


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-05,Condo/Co-op,9111 Collins Ave Unit N-611,Surfside,FL,33154.0,7800000.0,1.0,2.5,...,-80.121454,https://www.redfin.com/FL/Surfside/9111-Collin...,Jill Eber,Coldwell Banker Realty,None,None,Ximena Penuela,"Fort Realty, LLC",None,None
1,PAST SALE,2025-02-06,Condo/Co-op,5959 Collins Ave #1803,Miami Beach,FL,33140.0,6190000.0,4.0,4.5,...,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Jill Eber,Coldwell Banker Realty,Jill Hertzberg,Coldwell Banker Realty,Tracie Hamersley,Douglas Elliman,None,None
2,PAST SALE,2025-02-04,Condo/Co-op,788 NE 23rd St #5602,Miami,FL,33137.0,4500000.0,4.0,4.5,...,-80.185542,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,Nathan Shore,Douglas Elliman,Ben Jacobs,Douglas Elliman,Jason Zarco,BHHS EWM Realty,None,None
3,PAST SALE,2025-02-07,Condo/Co-op,1435 Brickell Ave #3501,Miami,FL,33131.0,2895000.0,2.0,2.5,...,-80.191943,https://www.redfin.com/FL/Miami/1435-Brickell-...,Nancy Batchelor,"Compass Florida, LLC.",None,None,Christian Dreyfuss,Brown Harris Stevens,None,None
4,PAST SALE,2025-02-04,Condo/Co-op,15901 Collins Ave #2506,Sunny Isles Beach,FL,33160.0,2090000.0,3.0,3.5,...,-80.121772,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Monica Zevel,"Julies Realty, LLC",None,None,Yotam Dahan,The Agency Florida LLC,Daniel Tzinker,The Agency Florida LLC
5,PAST SALE,2025-02-03,Condo/Co-op,17001 Collins Ave #4107,Sunny Isles Beach,FL,33160.0,2060000.0,2.0,3.0,...,-80.120941,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Carolina Lopez,One Sotheby's International Realty,None,None,Carolina Lopez,One Sotheby's International Realty,None,None
6,PAST SALE,2025-02-07,Condo/Co-op,1331 Brickell Bay Dr #4303,Miami,FL,33131.0,1900000.0,2.0,3.0,...,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Walter Faerman,Dezer Platinum Realty LLC,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None
7,PAST SALE,2025-02-03,Condo/Co-op,700 Coral Way #3,Coral Gables,FL,33134.0,1800000.0,3.0,3.0,...,-80.269144,https://www.redfin.com/FL/Coral-Gables/700-Cor...,William Ferrer,Universal 1 Realty,None,None,Maruja Gil,Keller Williams Capital Realty,None,None
8,PAST SALE,2025-02-06,Condo/Co-op,495 Brickell Ave #3101,Miami,FL,33131.0,1600000.0,3.0,2.0,...,-80.188672,https://www.redfin.com/FL/Miami/495-Brickell-A...,Martin Wischhusen,Global Luxury Realty LLC,None,None,Arianna Narciso,"Artis Real Estate Services, LLC.",None,None
9,PAST SALE,2025-02-03,Condo/Co-op,650 NE 32nd St #4001,Miami,FL,33137.0,1575000.0,4.0,4.5,...,-80.186249,https://www.redfin.com/FL/Miami/650-NE-32nd-St...,Adam Redolfi,Alure Capital,None,None,Adam Redolfi,Alure Capital,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

84


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$805,279


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$567


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$67,643,475


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
97
Input Previous Week Condo Average Sales Price:
733712
Input Previous Week Condo Average PSF:
525
Input Previous Week Condo Sales total (ex: 198_000_000)
93_900_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-05,Condo/Co-op,9111 Collins Ave Unit N-611,Surfside,FL,33154.0,7800000.0,1.0,2.5,SURF CLUB CONDO,2031.0,NaN,2017.0,NaN,3840.0,6827.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9111-Collin...,MARMLS,A11580902,N,Y,25.878108,-80.121454,https://www.redfin.com/FL/Surfside/9111-Collin...,Jill Eber,Coldwell Banker Realty,None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-05,Condo/Co-op,9111 Collins Ave Unit N-611,Surfside,FL,33154.0,7800000.0,1.0,2.5,SURF CLUB CONDO,2031.0,NaN,2017.0,NaN,3840.0,6827.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9111-Collin...,MARMLS,A11580902,N,Y,25.878108,-80.121454,https://www.redfin.com/FL/Surfside/9111-Collin...,Jill Eber,Coldwell Banker Realty,None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange
1,PAST SALE,2025-02-06,Condo/Co-op,5959 Collins Ave #1803,Miami Beach,FL,33140.0,6190000.0,4.0,4.5,BATH CLUB,3640.0,NaN,2005.0,NaN,1701.0,6710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11513873,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Jill Eber,Coldwell Banker Realty,Jill Hertzberg,Coldwell Banker Realty,Tracie Hamersley,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-02-04,Condo/Co-op,788 NE 23rd St #5602,Miami,FL,33137.0,4500000.0,4.0,4.5,NANCY TERRACE,3492.0,NaN,2021.0,NaN,1289.0,5650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,MARMLS,A11738937,N,Y,25.799065,-80.185542,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,Nathan Shore,Douglas Elliman,Ben Jacobs,Douglas Elliman,Jason Zarco,BHHS EWM Realty,None,None,3,blue
3,PAST SALE,2025-02-07,Condo/Co-op,1435 Brickell Ave #3501,Miami,FL,33131.0,2895000.0,2.0,2.5,MILLENNIUM TOWER CONDOMIN,1946.0,NaN,2003.0,NaN,1488.0,3954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1435-Brickell-...,MARMLS,A11715454,N,Y,25.759193,-80.191943,https://www.redfin.com/FL/Miami/1435-Brickell-...,Nancy Batchelor,"Compass Florida, LLC.",None,None,Christian Dreyfuss,Brown Harris Stevens,None,None,4,blue
4,PAST SALE,2025-02-04,Condo/Co-op,15901 Collins Ave #2506,Sunny Isles Beach,FL,33160.0,2090000.0,3.0,3.5,TDR TOWER II CONDO,2327.0,NaN,2008.0,NaN,898.0,5668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11694322,N,Y,25.923693,-80.121772,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Monica Zevel,"Julies Realty, LLC",None,None,Yotam Dahan,The Agency Florida LLC,Daniel Tzinker,The Agency Florida LLC,5,blue
5,PAST SALE,2025-02-03,Condo/Co-op,17001 Collins Ave #4107,Sunny Isles Beach,FL,33160.0,2060000.0,2.0,3.0,JADE BEACH CONDO,1479.0,NaN,2008.0,NaN,1393.0,1974.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11695000,N,Y,25.933531,-80.120941,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Carolina Lopez,One Sotheby's International Realty,None,None,Carolina Lopez,One Sotheby's International Realty,None,None,6,blue
6,PAST SALE,2025-02-07,Condo/Co-op,1331 Brickell Bay Dr #4303,Miami,FL,33131.0,1900000.0,2.0,3.0,JADE RESIDENCES AT BRICKE,1730.0,NaN,2004.0,NaN,1098.0,2375.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11666572,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Walter Faerman,Dezer Platinum Realty LLC,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,7,blue
7,PAST SALE,2025-02-03,Condo/Co-op,700 Coral Way #3,Coral Gables,FL,33134.0,1800000.0,3.0,3.0,GABLES BATH CLUB APTS CON,3200.0,NaN,1974.0,NaN,563.0,2346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/700-Cor...,MARMLS,A11526899,N,Y,25.748811,-80.269144,https://www.redfin.com/FL/Coral-Gables/700-Cor...,William Ferrer,Universal 1 Realty,None,None,Maruja Gil,Keller Williams Capital Realty,None,None,8,blue
8,PAST SALE,2025-02-06,Condo/Co-op,495 Brickell Ave #3101,Miami,FL,33131.0,1600000.0,3.0,2.0,ICON BRICKELL NO TWO,1870.0,NaN,2009.0,NaN,856.0,2229.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/49

In [32]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [33]:
m.save('index.html')

## Data snagger

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-05,Condo/Co-op,9111 Collins Ave Unit N-611,Surfside,FL,33154.0,7800000.0,1.0,2.5,SURF CLUB CONDO,2031.0,NaN,2017.0,NaN,3840.0,6827.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9111-Collin...,MARMLS,A11580902,N,Y,25.878108,-80.121454,https://www.redfin.com/FL/Surfside/9111-Collin...,Jill Eber,Coldwell Banker Realty,None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_02102024


# CREATE TEMPLATE 

In [37]:
muni_set = set(df_top_ten['CITY'])

In [38]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [39]:
df_top_ten.reset_index(inplace=True,drop=True)

In [40]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [41]:
top_sale

'SURF CLUB CONDO at 9111 Collins Ave Unit N-611 in Surfside'

In [42]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [43]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-05,Condo/Co-op,9111 Collins Ave Unit N-611,Surfside,FL,33154.0,7800000.0,1.0,2.5,SURF CLUB CONDO,2031.0,NaN,2017.0,NaN,3840.0,6827.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9111-Collin...,MARMLS,A11580902,N,Y,25.878108,-80.121454,https://www.redfin.com/FL/Surfside/9111-Collin...,Jill Eber,Coldwell Banker Realty,None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9111 Collins Ave Unit N-611...
1,PAST SALE,2025-02-06,Condo/Co-op,5959 Collins Ave #1803,Miami Beach,FL,33140.0,6190000.0,4.0,4.5,BATH CLUB,3640.0,NaN,2005.0,NaN,1701.0,6710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11513873,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Jill Eber,Coldwell Banker Realty,Jill Hertzberg,Coldwell Banker Realty,Tracie Hamersley,Douglas Elliman,None,None,2,blue,BATH CLUB at 5959 Collins Ave #1803 in Miami B...
2,PAST SALE,2025-02-04,Condo/Co-op,788 NE 23rd St #5602,Miami,FL,33137.0,4500000.0,4.0,4.5,NANCY TERRACE,3492.0,NaN,2021.0,NaN,1289.0,5650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,MARMLS,A11738937,N,Y,25.799065,-80.185542,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,Nathan Shore,Douglas Elliman,Ben Jacobs,Douglas Elliman,Jason Zarco,BHHS EWM Realty,None,None,3,blue,NANCY TERRACE at 788 NE 23rd St #5602 in Miami
3,PAST SALE,2025-02-07,Condo/Co-op,1435 Brickell Ave #3501,Miami,FL,33131.0,2895000.0,2.0,2.5,MILLENNIUM TOWER CONDOMIN,1946.0,NaN,2003.0,NaN,1488.0,3954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1435-Brickell-...,MARMLS,A11715454,N,Y,25.759193,-80.191943,https://www.redfin.com/FL/Miami/1435-Brickell-...,Nancy Batchelor,"Compass Florida, LLC.",None,None,Christian Dreyfuss,Brown Harris Stevens,None,None,4,blue,MILLENNIUM TOWER CONDOMIN at 1435 Brickell Ave...
4,PAST SALE,2025-02-04,Condo/Co-op,15901 Collins Ave #2506,Sunny Isles Beach,FL,33160.0,2090000.0,3.0,3.5,TDR TOWER II CONDO,2327.0,NaN,2008.0,NaN,898.0,5668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11694322,N,Y,25.923693,-80.121772,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Monica Zevel,"Julies Realty, LLC",None,None,Yotam Dahan,The Agency Florida LLC,Daniel Tzinker,The Agency Florida LLC,5,blue,TDR TOWER II CONDO at 15901 Collins Ave #2506 ...
5,PAST SALE,2025-02-03,Condo/Co-op,17001 Collins Ave #4107,Sunny Isles Beach,FL,33160.0,2060000.0,2.0,3.0,JADE BEACH CONDO,1479.0,NaN,2008.0,NaN,1393.0,1974.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11695000,N,Y,25.933531,-80.120941,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Carolina Lopez,One Sotheby's International Realty,None,None,Carolina Lopez,One Sotheby's International Realty,None,None,6,blue,JADE BEACH CONDO at 17001 Collins Ave #4107 in...
6,PAST SALE,2025-02-07,Condo/Co-op,1331 Brickell Bay Dr #4303,Miami,FL,33131.0,1900000.0,2.0,3.0,JADE RESIDENCES AT BRICKE,1730.0,NaN,2004.0,NaN,1098.0,2375.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11666572,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Walter Faerman,Dezer Platinum Realty LLC,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,7,blue,JADE RESIDENCES AT BRICKE at 1331 Brickell Bay...
7,PAST SALE,2025-02-03,Condo/Co-op,700 Coral Way #3,Coral Gables,FL,33134.0,1800000.0,3.0,3.0,GABLES BATH CLUB APTS CON,3200.0,NaN,1974.0,NaN,563.0,2346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/700-Cor...,MARMLS,A11526899,N,Y,25.748811,-80

In [45]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: SURF CLUB CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,575,000 to $7,800,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Surfside, Miami Beach, Miami, Coral Gables, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 84 condo sales totaling $67,643,475 million from Feb. 2nd to Feb. 9th. The previous week, brokers closed 97 condo sales totaling $93,900,000.

Last week’s units sold for an average of $805,279, higher than the $733,712 average price from the previous week. The 

In [46]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [47]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-05,Condo/Co-op,9111 Collins Ave Unit N-611,Surfside,FL,33154.0,7800000.0,1.0,2.5,SURF CLUB CONDO,2031.0,NaN,2017.0,NaN,3840.0,6827.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9111-Collin...,MARMLS,A11580902,N,Y,25.878108,-80.121454,https://www.redfin.com/FL/Surfside/9111-Collin...,Jill Eber,Coldwell Banker Realty,None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9111 Collins Ave Unit N-611...
1,PAST SALE,2025-02-06,Condo/Co-op,5959 Collins Ave #1803,Miami Beach,FL,33140.0,6190000.0,4.0,4.5,BATH CLUB,3640.0,NaN,2005.0,NaN,1701.0,6710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11513873,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Jill Eber,Coldwell Banker Realty,Jill Hertzberg,Coldwell Banker Realty,Tracie Hamersley,Douglas Elliman,None,None,2,blue,BATH CLUB at 5959 Collins Ave #1803 in Miami B...
2,PAST SALE,2025-02-04,Condo/Co-op,788 NE 23rd St #5602,Miami,FL,33137.0,4500000.0,4.0,4.5,NANCY TERRACE,3492.0,NaN,2021.0,NaN,1289.0,5650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,MARMLS,A11738937,N,Y,25.799065,-80.185542,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,Nathan Shore,Douglas Elliman,Ben Jacobs,Douglas Elliman,Jason Zarco,BHHS EWM Realty,None,None,3,blue,NANCY TERRACE at 788 NE 23rd St #5602 in Miami
3,PAST SALE,2025-02-07,Condo/Co-op,1435 Brickell Ave #3501,Miami,FL,33131.0,2895000.0,2.0,2.5,MILLENNIUM TOWER CONDOMIN,1946.0,NaN,2003.0,NaN,1488.0,3954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1435-Brickell-...,MARMLS,A11715454,N,Y,25.759193,-80.191943,https://www.redfin.com/FL/Miami/1435-Brickell-...,Nancy Batchelor,"Compass Florida, LLC.",None,None,Christian Dreyfuss,Brown Harris Stevens,None,None,4,blue,MILLENNIUM TOWER CONDOMIN at 1435 Brickell Ave...
4,PAST SALE,2025-02-04,Condo/Co-op,15901 Collins Ave #2506,Sunny Isles Beach,FL,33160.0,2090000.0,3.0,3.5,TDR TOWER II CONDO,2327.0,NaN,2008.0,NaN,898.0,5668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11694322,N,Y,25.923693,-80.121772,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Monica Zevel,"Julies Realty, LLC",None,None,Yotam Dahan,The Agency Florida LLC,Daniel Tzinker,The Agency Florida LLC,5,blue,TDR TOWER II CONDO at 15901 Collins Ave #2506 ...
5,PAST SALE,2025-02-03,Condo/Co-op,17001 Collins Ave #4107,Sunny Isles Beach,FL,33160.0,2060000.0,2.0,3.0,JADE BEACH CONDO,1479.0,NaN,2008.0,NaN,1393.0,1974.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11695000,N,Y,25.933531,-80.120941,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Carolina Lopez,One Sotheby's International Realty,None,None,Carolina Lopez,One Sotheby's International Realty,None,None,6,blue,JADE BEACH CONDO at 17001 Collins Ave #4107 in...
6,PAST SALE,2025-02-07,Condo/Co-op,1331 Brickell Bay Dr #4303,Miami,FL,33131.0,1900000.0,2.0,3.0,JADE RESIDENCES AT BRICKE,1730.0,NaN,2004.0,NaN,1098.0,2375.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11666572,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Walter Faerman,Dezer Platinum Realty LLC,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,7,blue,JADE RESIDENCES AT BRICKE at 1331 Brickell Bay...
7,PAST SALE,2025-02-03,Condo/Co-op,700 Coral Way #3,Coral Gables,FL,33134.0,1800000.0,3.0,3.0,GABLES BATH CLUB APTS CON,3200.0,NaN,1974.0,NaN,563.0,2346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/700-Cor...,MARMLS,A11526899,N,Y,25.748811,-80

In [50]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: SURF CLUB CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,575,000 to $7,800,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Surfside, Miami Beach, Miami, Coral Gables, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 84 condo sales totaling $67,643,475 million from Feb. 2nd to Feb. 9th. The previous week, brokers closed 97 condo sales totaling $93,900,000.

Last week’s units sold for an average of $805,279, higher than the $733,712 average price from the previous week. The 

In [51]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Surfside/9111-Collins-Ave-33154/unit-N611/home/79133148


In [52]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/5959-Collins-Ave-33140/unit-1803/home/43339298


In [53]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/650-NE-32nd-St-33137/unit-4001/home/56745331


In [54]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/495-Brickell-Ave-33131/unit-3101/home/43408362


In [55]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Surfside/9111-Collins-Ave-33154/unit-N611/home/79133148


In [56]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Coral-Gables/700-Coral-Way-33134/unit-3/home/42804565


In [57]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-05,Condo/Co-op,9111 Collins Ave Unit N-611,Surfside,FL,33154.0,7800000.0,1.0,2.5,SURF CLUB CONDO,2031.0,NaN,2017.0,NaN,3840.0,6827.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9111-Collin...,MARMLS,A11580902,N,Y,25.878108,-80.121454,https://www.redfin.com/FL/Surfside/9111-Collin...,Jill Eber,Coldwell Banker Realty,None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9111 Collins Ave Unit N-611...
1,PAST SALE,2025-02-06,Condo/Co-op,5959 Collins Ave #1803,Miami Beach,FL,33140.0,6190000.0,4.0,4.5,BATH CLUB,3640.0,NaN,2005.0,NaN,1701.0,6710.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11513873,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Jill Eber,Coldwell Banker Realty,Jill Hertzberg,Coldwell Banker Realty,Tracie Hamersley,Douglas Elliman,None,None,2,blue,BATH CLUB at 5959 Collins Ave #1803 in Miami B...
2,PAST SALE,2025-02-04,Condo/Co-op,788 NE 23rd St #5602,Miami,FL,33137.0,4500000.0,4.0,4.5,NANCY TERRACE,3492.0,NaN,2021.0,NaN,1289.0,5650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,MARMLS,A11738937,N,Y,25.799065,-80.185542,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,Nathan Shore,Douglas Elliman,Ben Jacobs,Douglas Elliman,Jason Zarco,BHHS EWM Realty,None,None,3,blue,NANCY TERRACE at 788 NE 23rd St #5602 in Miami
3,PAST SALE,2025-02-07,Condo/Co-op,1435 Brickell Ave #3501,Miami,FL,33131.0,2895000.0,2.0,2.5,MILLENNIUM TOWER CONDOMIN,1946.0,NaN,2003.0,NaN,1488.0,3954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1435-Brickell-...,MARMLS,A11715454,N,Y,25.759193,-80.191943,https://www.redfin.com/FL/Miami/1435-Brickell-...,Nancy Batchelor,"Compass Florida, LLC.",None,None,Christian Dreyfuss,Brown Harris Stevens,None,None,4,blue,MILLENNIUM TOWER CONDOMIN at 1435 Brickell Ave...
4,PAST SALE,2025-02-04,Condo/Co-op,15901 Collins Ave #2506,Sunny Isles Beach,FL,33160.0,2090000.0,3.0,3.5,TDR TOWER II CONDO,2327.0,NaN,2008.0,NaN,898.0,5668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11694322,N,Y,25.923693,-80.121772,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Monica Zevel,"Julies Realty, LLC",None,None,Yotam Dahan,The Agency Florida LLC,Daniel Tzinker,The Agency Florida LLC,5,blue,TDR TOWER II CONDO at 15901 Collins Ave #2506 ...
5,PAST SALE,2025-02-03,Condo/Co-op,17001 Collins Ave #4107,Sunny Isles Beach,FL,33160.0,2060000.0,2.0,3.0,JADE BEACH CONDO,1479.0,NaN,2008.0,NaN,1393.0,1974.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11695000,N,Y,25.933531,-80.120941,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Carolina Lopez,One Sotheby's International Realty,None,None,Carolina Lopez,One Sotheby's International Realty,None,None,6,blue,JADE BEACH CONDO at 17001 Collins Ave #4107 in...
6,PAST SALE,2025-02-07,Condo/Co-op,1331 Brickell Bay Dr #4303,Miami,FL,33131.0,1900000.0,2.0,3.0,JADE RESIDENCES AT BRICKE,1730.0,NaN,2004.0,NaN,1098.0,2375.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11666572,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Walter Faerman,Dezer Platinum Realty LLC,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,7,blue,JADE RESIDENCES AT BRICKE at 1331 Brickell Bay...
7,PAST SALE,2025-02-03,Condo/Co-op,700 Coral Way #3,Coral Gables,FL,33134.0,1800000.0,3.0,3.0,GABLES BATH CLUB APTS CON,3200.0,NaN,1974.0,NaN,563.0,2346.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/700-Cor...,MARMLS,A11526899,N,Y,25.748811,-80

## Time on Market Calculator

In [59]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 12, 19) ## List (Earlier) date
date2 = datetime(2025, 2, 6) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

415


## Clean CSV for Datawrapper Chart

In [60]:
chart_df = df_top_ten

In [61]:
chart_df = chart_df.fillna(" ")

In [62]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [63]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [64]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [65]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [66]:
chart_df['$/SQUARE FEET'].astype(float)

0    3840.0
1    1701.0
2    1289.0
3    1488.0
4     898.0
5    1393.0
6    1098.0
7     563.0
8     856.0
9     841.0
Name: $/SQUARE FEET, dtype: float64

In [67]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [68]:
csv_date_string = today.strftime("%m_%d_%Y")

In [69]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [70]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-05,9111 Collins Ave Unit N-611 in Surfside,"$7,800,000",1,2.5,"2,031",2017,"$3,840",MARMLS,A11580902,Jill Eber Coldwell Banker Realty,"Ximena Penuela Fort Realty, LLC"
1,2025-02-06,5959 Collins Ave #1803 in Miami Beach,"$6,190,000",4,4.5,"3,640",2005,"$1,701",MARMLS,A11513873,Jill Eber Coldwell Banker Realty Jill Hertzber...,Tracie Hamersley Douglas Elliman
2,2025-02-04,788 NE 23rd St #5602 in Miami,"$4,500,000",4,4.5,"3,492",2021,"$1,289",MARMLS,A11738937,Nathan Shore Douglas Elliman Ben Jacobs Dougla...,Jason Zarco BHHS EWM Realty
3,2025-02-07,1435 Brickell Ave #3501 in Miami,"$2,895,000",2,2.5,"1,946",2003,"$1,488",MARMLS,A11715454,"Nancy Batchelor Compass Florida, LLC.",Christian Dreyfuss Brown Harris Stevens
4,2025-02-04,15901 Collins Ave #2506 in Sunny Isles Beach,"$2,090,000",3,3.5,"2,327",2008,$898,MARMLS,A11694322,"Monica Zevel Julies Realty, LLC",Yotam Dahan The Agency Florida LLC Daniel Tzin...
5,2025-02-03,17001 Collins Ave #4107 in Sunny Isles Beach,"$2,060,000",2,3.0,"1,479",2008,"$1,393",MARMLS,A11695000,Carolina Lopez One Sotheby's International Rea...,Carolina Lopez One Sotheby's International Rea...
6,2025-02-07,1331 Brickell Bay Dr #4303 in Miami,"$1,900,000",2,3.0,"1,730",2004,"$1,098",MARMLS,A11666572,Walter Faerman Dezer Platinum Realty LLC,NON-MLS MEMBER MAR NON MLS MEMBER
7,2025-02-03,700 Coral Way #3 in Coral Gables,"$1,800,000",3,3.0,"3,200",1974,$563,MARMLS,A11526899,William Ferrer Universal 1 Realty,Maruja Gil Keller Williams Capital Realty
8,2025-02-06,495 Brickell Ave #3101 in Miami,"$1,600,000",3,2.0,"1,870",2009,$856,MARMLS,A11668802,Martin Wischhusen Global Luxury Realty LLC,"Arianna Narciso Artis Real Estate Services, LL..."
9,2025-02-03,650 NE 32nd St #4001 in Miami,"$1,575,000",4,4.5,"1,872",2018,$841,MARMLS,A11664569,Adam Redolfi Alure Capital,Adam Redolfi Alure Capital
